<a href="https://colab.research.google.com/github/KwinLook/ADEX-Project/blob/main/ADEX_Project_Launch_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import os
import cv2
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from os import listdir
from os.path import isfile, join

In [ ]:
train_dir = 'train/'
validation_dir = 'validation/'
num_classes =2
width = 150

In [ ]:
train_Explosive_dir = os.path.join(train_dir, 'explosive')  # directory with our training Explosive pictures
train_None_dir = os.path.join(train_dir, 'none')  # directory with our training Non-Explosiv pictures
validation_Explosive_dir = os.path.join(validation_dir, 'explosive')  # directory with our validation Explosive pictures
validation_None_dir = os.path.join(validation_dir, 'none')  # directory with our validation Non-Explosiv pictures

num_Explosive_tr = len(os.listdir(train_Explosive_dir))
num_None_tr = len(os.listdir(train_None_dir))

num_Explosive_val = len(os.listdir(validation_Explosive_dir))
num_None_val = len(os.listdir(validation_None_dir))

total_train = num_Explosive_tr + num_None_tr
total_val = num_Explosive_val + num_None_val

In [ ]:
trainImg = [train_dir+f for f in listdir(train_dir) if listdir(join(train_dir, f))]
valImg = [validation_dir+f for f in listdir(validation_dir) if listdir(join(validation_dir, f))]

In [ ]:
# One-Hot Encoding
def img2data(path):
  rawImgs = []
  labels = []
  c = 0
  for imagePath in (path):
      for item in tqdm(os.listdir(imagePath)):
          file = os.path.join(imagePath, item)
          #print(file)
          c+=1
          l = imagePath.split('/')[1]
          if l == 'explosive':
            labels.append([1,0])         
          elif l == 'none':
            labels.append([0,1])
          img = cv2.imread(file , cv2.COLOR_BGR2RGB)
          img = cv2.resize(img ,(width,width))
          rawImgs.append(img)
  return rawImgs, labels

In [ ]:
x_train, y_train = img2data(trainImg)
x_val, y_val = img2data(valImg)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255

In [ ]:
x_train.shape,y_train.shape,x_val.shape, y_val.shape

In [ ]:
#Create Nueral Network Test Dropout
model = keras.Sequential([
        keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(width, width, 3)),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(0.75),
        keras.layers.Conv2D(100,(3,3) , activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2 )),
        keras.layers.Dropout(0.5),
        keras.layers.Conv2D(64,(3,3) , activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2 )),
        keras.layers.Dropout(0.25),
        keras.layers.Conv2D(32,(3,3) , activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2 )),
        keras.layers.Dense(16),
        keras.layers.Flatten(),
        keras.layers.Dense(2),
        keras.layers.Dense(num_classes, activation='softmax') #softmax for one hot . . # sigmoid for 0/1
    ])


In [ ]:
model.summary()

In [ ]:

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=100
batch_size =35
history = model.fit(x_train, y_train ,batch_size=batch_size, epochs=epochs ,validation_data=(x_val, y_val))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
predict_dir = 'generated_predict/'
predictImg = [predict_dir+f for f in listdir(predict_dir) if listdir(join(predict_dir, f))]
rimg = []
for imagePath in (predictImg):
    for item in (os.listdir(imagePath)):
        file = os.path.join(imagePath, item)
        if item.split('.')[0] != "":
           
          img = cv2.imread(file , cv2.COLOR_BGR2RGB)
          ori = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img = cv2.resize(img ,(width,width))
          rimg = np.array(img)
          rimg = rimg.astype('float32')
          rimg /= 255
          rimg = np.reshape(rimg ,(1,150,150,3))
          predict = model.predict(rimg)
          label = ['explosive','none']
          result = label[np.argmax(predict)]
          print(predict)
          print('real:'+str(item))
          print('predict:'+str(result))
          #cv2.putText(img, 'real:'+str(item), (10,10), cv2.FONT_HERSHEY_SIMPLEX, 0.2, (255,0,))
          #cv2.putText(img, 'predict'+str(result), (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.2, (255,0,0))
          plt.imshow(ori)
          plt.show()